# Sample Code

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04
Err:1 http://security.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04
  404  Not Found [IP: 91.189.88.152 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.152 80]
E: Unable to fetch some archives, maybe run apt-get update o

In [2]:
from pyspark.sql import SparkSession, DataFrame, Window
from pyspark.sql.functions import col, regexp_extract, regexp_replace, lit, when
import pyspark.sql.functions as func
from pyspark.sql.types import *
from pyspark.sql.utils import AnalysisException
from pyspark import StorageLevel

import sys
from datetime import datetime, timedelta
from functools import reduce

In [3]:
spark = SparkSession.builder \
    .appName("content-based") \
    .config("spark.sql.files.ignoreCorruptFiles", "true") \
    .config("spark.sql.session.timeZone", "Asia/Taipei") \
    .getOrCreate()

## 基礎建設

In [4]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [5]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-25 13:48:26--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.2’

All_Beauty.csv.2    100%[===================>]  14.78M  60.6MB/s    in 0.2s    

2021-12-25 13:48:26 (60.6 MB/s) - ‘All_Beauty.csv.2’ saved [15499476/15499476]

--2021-12-25 13:48:26--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.2’

meta_All_Beauty.jso 100%[===================>]   9.85M  53.6MB/s    in 0.2s   

In [6]:
# schema_rating = StructType([
#                     StructField("asin", StringType())
#                       , StructField("reviewerID", StringType())
#                       , StructField("overall", StringType())
#                       , StructField("unixReviewTime", StringType())
#                       ])

metadata = spark.read.json("file:////content/meta_All_Beauty.json.gz")
# ratings = spark.read.format("csv").option("header", True).schema(schema_rating).load("file:///content/All_Beauty.csv")

# metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
# metadata.head()
# ratings.head()

## 資料整理

In [7]:
metadata = metadata.select(
                col("asin")
                , col("title")
                , func.when(col("title") == '', 0).otherwise(1).alias("if_title")
                , col("feature")
                , func.when(func.size("feature") == 0, 0).otherwise(1).alias("if_feature")
                , col("description")
                , func.when(func.size("description") == 0, 0).otherwise(1).alias("if_description")
                , func.translate(col("price"), ',$', '').cast(DoubleType()).alias("price")
                , col("imageURL"), col("imageURLHighRes")
                , func.when(func.size("imageURL") == 0, 0).otherwise(1).alias("if_imageURL")
                , func.when(func.size("imageURLHighRes") == 0, 0).otherwise(1).alias("if_imageURLHighRes")
                , col("also_buy"), col("also_view")
                , func.translate(func.split(col("rank"), ' in ')[0], ',', '').cast(IntegerType()).alias("rank_sub_category")
                , func.translate(func.split(col("rank"), ' in ')[1], '()"];', '').alias("sub_category")
                , col("brand")
                , col("tech1")
                , func.when(func.length("tech1") == 0, 0).otherwise(1).alias("if_tech1")
                , col("similar_item"), col("date")
                , col("details.\n    Item Weight: \n    ").alias("item_weight")
                , col("details.\n    Product Dimensions: \n    ").alias("product_dimensions")
                , col("details.Batteries").alias("batteries")
                , func.when(col("details.Batteries").isNull(), 0).otherwise(1).alias("if_batteries")
                , col("details.Discontinued by manufacturer:").alias("discountedByManufacturer")
                , func.when(col("details.Discontinued by manufacturer:").isNull(), 0).otherwise(1).alias("if_discountedByManufacturer")
                , col("details.Domestic Shipping: ").alias("domestic_shipping")
                , col("details.International Shipping: ").alias("international_shipping")
                , col("details.Item model number:").alias("item_model_no")
                , col("details.Shipping Advisory:").alias("shipping_advisory")
                , col("details.Shipping Weight:").alias("shipping_weight")
                )

In [18]:
# if the info on below variables are not null, then sum as 1
# log10 rank by sub_category and multiply 100 as score
metadata_scored = metadata.withColumn('score_if', func.sum(col("if_title")+col("if_feature")+col("if_description")+col("if_imageURL") \
                                      +col("if_imageURLHighRes")+col("if_tech1")+col("if_batteries") \
                                      +col("if_discountedByManufacturer")).over(Window.partitionBy("asin"))) \
                   .withColumn('score_rank', (100 / func.log10(col("rank_sub_category"))).alias("score_rank")) \
                   .withColumn('score_product', col("score_if") + col("score_rank"))
metadata_selected = metadata_scored.select("asin", "brand", "sub_category", "score_product")

In [19]:
# metadata.stat.corr("rank_sub_category", "score_if")
metadata_scored.summary().show(10, False)
metadata_selected.summary().show(10, False)

+-------+--------------------+---------------------------------------------------------------------+--------------------+-------------------+-------------------+------------------+------------------+------------------+-----------------+-------------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
metadata_pd = metadata_selected.toPandas()

In [23]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
# ratings = ratings.select("asin", "reviewerID", col("overall").cast(DoubleType()).alias("overall")
#               , func.from_unixtime("unixReviewTime","yyyy-MM-dd HH:mm:ss").alias("timestamp"))

In [26]:
ratings_metadata = pd.merge(ratings,metadata_pd,on='asin',how='left')
ratings_metadata.head(2)

,asin,reviewerID,overall,unixReviewTime,DATE,brand,sub_category,score_product
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19,NaN,NaN,NaN
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18,NaN,NaN,NaN


In [60]:
ratings_metadata['brand_popularity'] = ratings_metadata.groupby(["reviewerID", "brand"])["asin"].transform('count')
ratings_metadata['brand_popularity'].fillna(1, inplace=True)
ratings_metadata['subcategory_popularity'] = ratings_metadata.groupby(["reviewerID", "sub_category"])["asin"].transform('count')
ratings_metadata['subcategory_popularity'].fillna(1, inplace=True)
ratings_metadata['product_popularity'] = ratings_metadata.groupby(["asin"])["reviewerID"].transform('count')
ratings_metadata.head(2)

,asin,reviewerID,overall,unixReviewTime,DATE,brand,sub_category,score_product,brand_popularity,subcategory_popularity,product_popularity
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19,NaN,NaN,NaN,1.0,1.0,17
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18,NaN,NaN,NaN,1.0,1.0,17


In [64]:
ratings_metadata.describe()

,overall,unixReviewTime,score_product,brand_popularity,subcategory_popularity,product_popularity
count,387654.000000,3.876540e+05,378067.000000,387654.000000,387654.000000,387654.000000
mean,4.118012,1.438586e+09,23.629657,1.244739,1.378497,900.689311
std,1.358516,7.318079e+07,6.197040,0.551563,0.966107,1945.473119
min,1.000000,9.474624e+08,16.286408,1.000000,1.000000,1.000000
25%,4.000000,1.413936e+09,20.216321,1.000000,1.000000,20.000000
50%,5.000000,1.456186e+09,21.881549,1.000000,1.000000,114.000000
75%,5.000000,1.485302e+09,25.039307,1.000000,1.000000,589.000000
max,5.000000,1.538438e+09,68.763985,13.000000,26.000000,8672.000000


## 資料切分

In [63]:
ratings_trainings = ratings_metadata[
    (ratings_metadata['DATE'] < '2018-09-01')
]
ratings_testings = ratings_metadata[
    (ratings_metadata['DATE'] >= '2018-09-01') & 
    (ratings_metadata['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [65]:
{user: ratings_trainings['asin'].sample(n=2).tolist() for user in users}

{'A100XQFWKQ30O2': ['B003CU26SY', 'B01ES8NA38'],
 'A103T1QOGFCSEH': ['B01GYFO0DM', 'B001DKQ47E'],
 'A106UKKSJ2KXPF': ['B001BROT1M', 'B00X3Q81SQ'],
 'A10A7GV4D5A11V': ['B00005JS5C', 'B01E5GSPWE'],
 'A1119JJ37ZLB8R': ['B00SKYNQM2', 'B00S4ADDFW'],
 'A113UOOLBSZN52': ['B001EODA2G', 'B00GIX3TO2'],
 'A12M4U7WK4ALCR': ['B000GLRREU', 'B01BHM6K1C'],
 'A12T8YTW6VWT7S': ['B00EYHM70G', 'B00PMRE5A2'],
 'A1364JXGKB46MM': ['B001FB5H9C', 'B000NCTTGG'],
 'A137DALOQFKBTI': ['B00JG8DY0A', 'B00INX0TMA'],
 'A13FEZ3WV7S2EY': ['B00GH7OSEO', 'B01G6SL6IO'],
 'A13IV4I1B0RXMG': ['B01CS89CQY', 'B01GQ85WAC'],
 'A13JU88JAHN72I': ['B00005JS5C', 'B00K08R11I'],
 'A13K55R6VH1OOD': ['B000209JS2', 'B01AVEON1Q'],
 'A13P7VFU075A': ['B001P2NRPC', 'B01016RFCS'],
 'A13SWYE4QLB6NG': ['1620213982', 'B00X4DKZKU'],
 'A13ZTQ0Q4ATA41': ['B000FOI48G', 'B006WYJM8Y'],
 'A142EDN04OD62U': ['B01DVW54I6', 'B0085RZLVS'],
 'A142I22FIC8MZK': ['B00005JS5C', 'B00H1M8R3C'],
 'A14834QTII5TLT': ['B015AWTL5M', 'B0091OCDRE'],
 'A14A447VPACTBC': ['B

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    ruled-based
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations


ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

In [ ]:
def recommender_random(training_data, users=[], k=2):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    random-based
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations
ratings_by_user = recommender_random(ratings_trainings, users)
ratings_by_user